In [13]:
# Log and register the model IrisRandomForestModel in MLflow (data from Iris)
from sklearn.datasets import load_iris
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

mlflow.set_tracking_uri("http://localhost:5000")

iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier()
model.fit(X_train, y_train)

with mlflow.start_run() as run:
    mlflow.sklearn.log_model(model, "random_forest_model", registered_model_name="IrisRandomForestModel")
    mlflow.log_params({"n_estimators": 100, "random_state": 42})
    mlflow.log_metric("accuracy", model.score(X_test, y_test))
    print("Model saved in run %s" % run.info.run_uuid)


Successfully registered model 'IrisRandomForestModel'.
2024/07/25 12:59:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IrisRandomForestModel, version 1


Model saved in run ad35a33723ab43a1b102b15d01c1ae69


Created version '1' of model 'IrisRandomForestModel'.


In [14]:
# testing the model
import mlflow
import mlflow.sklearn

mlflow.set_tracking_uri("http://localhost:5000")

# Fetch the latest model version
client = mlflow.tracking.MlflowClient()
latest_run = client.search_runs(
    experiment_ids=["0"], order_by=["attribute.start_time DESC"], max_results=1
)[0]

model_uri = f"runs:/{latest_run.info.run_id}/random_forest_model"
model = mlflow.sklearn.load_model(model_uri)

print(model.predict([[4.1, 4.2, 3.4, 0.2]]))
print(model.predict([[1.1, 3.0, 2.4, 1.2]]))
print(model.predict([[3.1, 2.3, 4.4, 4.2]]))
print(model.predict([[2.1, 1.8, 1.4, 2.2]]))


[0]
[1]
[2]
[0]


In [17]:
# testing diff versions
import mlflow.pyfunc

mlflow.set_tracking_uri("http://localhost:5000")

model_name = "IrisRandomForestModel"
model_version = 1  # Specify the version number
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

input_features = [[5.1, 3.5, 1.4, 0.2]]
print(model.predict(input_features))


[0]


In [18]:
# promoting model to production
from mlflow.tracking import MlflowClient

client = MlflowClient()
client.transition_model_version_stage(
    name="IrisRandomForestModel",
    version=1,
    stage="Production"
)


C:\Users\user123\AppData\Local\Temp\ipykernel_16500\2902670911.py:4: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1721901561457, current_stage='Production', description='', last_updated_timestamp=1721901784385, name='IrisRandomForestModel', run_id='ad35a33723ab43a1b102b15d01c1ae69', run_link='', source=('file:///C:/Users/user123/Desktop/SEfactory/week-5/AI and '
 'ML/project/mlruns/0/ad35a33723ab43a1b102b15d01c1ae69/artifacts/random_forest_model'), status='READY', status_message='', tags={}, user_id='', version='1'>

In [19]:
# creating a new versrion

from sklearn.datasets import load_iris
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

mlflow.set_tracking_uri("http://localhost:5000")

iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=200)  # Changed from 100 to 200
model.fit(X_train, y_train)

with mlflow.start_run() as run:
    mlflow.sklearn.log_model(model, "random_forest_model", registered_model_name="IrisRandomForestModel")
    mlflow.log_params({"n_estimators": 200, "random_state": 42})
    mlflow.log_metric("accuracy", model.score(X_test, y_test))
    print("Model saved in run %s" % run.info.run_uuid)


Registered model 'IrisRandomForestModel' already exists. Creating a new version of this model...
2024/07/25 13:08:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IrisRandomForestModel, version 2


Model saved in run 35edf771a8e649ec98410ee067269679


Created version '2' of model 'IrisRandomForestModel'.
